In [1]:
!pip list

Package                   Version
------------------------- -----------
aiohttp                   3.9.5
aiosignal                 1.3.1
altair                    5.3.0
annotated-types           0.6.0
anyio                     4.3.0
asttokens                 2.0.5
async-timeout             4.0.3
attrs                     23.2.0
backcall                  0.2.0
backoff                   2.2.1
beautifulsoup4            4.12.3
blinker                   1.7.0
cachetools                5.3.3
certifi                   2024.2.2
cffi                      1.16.0
chardet                   5.2.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.1.2
dataclasses-json          0.6.4
debugpy                   1.6.7
decorator                 5.1.1
deepdiff                  7.0.1
distro                    1.9.0
emoji                     2.11.1
exceptiongroup            1.0.4
executing                 0.8.3
faiss-cpu                

# Prompt Engineering Playground

In [81]:
from dotenv import load_dotenv
import os
import json
from typing import List
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
load_dotenv()


True

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model=os.getenv("MODEL_NAME"),
    temperature=0,
    max_tokens=2000,
    timeout=None,
    max_retries=2,
    # other params...
)

In [15]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je adore programmer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 55, 'total_tokens': 60, 'completion_time': 0.006666667, 'prompt_time': 0.011249524, 'queue_time': 0.002200064, 'total_time': 0.017916191}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a60594f-97c5-4491-aa69-79d9a349a54a-0', usage_metadata={'input_tokens': 55, 'output_tokens': 5, 'total_tokens': 60})

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
response = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

print(response.content)

Ich liebe Programmieren.


In [36]:
def get_chatbot_response(model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append(message)
    
    llm = ChatGroq(
        model=os.getenv("MODEL_NAME",model_name) ,
        temperature=temperature,
        max_tokens=2000,
        timeout=None,
        max_retries=2,
    )
    
    response = llm.invoke(input_messages)
    
    return response.content
    
    
    

## LLM Answer Generation

In [ ]:
messages = [("system","what is the capital of germany")]
answer = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)

answer

[('system', 'what is the capital of germany'), ('system', 'what is the capital of france?')]
content='The capital of Germany is Berlin.\nThe capital of France is Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 44, 'total_tokens': 59, 'completion_time': 0.02, 'prompt_time': 0.009101247, 'queue_time': 0.005340892999999999, 'total_time': 0.029101247}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None} id='run-4e310a21-7e16-41af-8b9a-3b7df503d3ad-0' usage_metadata={'input_tokens': 44, 'output_tokens': 15, 'total_tokens': 59}


'The capital of Germany is Berlin.\nThe capital of France is Paris.'

## Structured Output

In [37]:
system_prompt = """
    You are a helpfull assistant the answer the questions about capitals of countries.
    
    Your output should be in a structured json format exactly like the one below.You are not allowed to write anything other than the json object.
    
    [
        {
            "country":the country that you will get the acapital of,
            "capital":the capital of the country stated
        }
    ]

"""


messages = [("system",system_prompt)]
messages.append(("human","what is the capital of Germany"))
messages.append(("human","what is the capital of France"))

answer = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)

print(answer)

[
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [42]:
json_response = json.loads(answer)
json_response[0]['capital']

'Paris'

## Input Structuring

In [43]:
user_input ="""
    Get me the capitals of the following countries:
    '''
    1.Italy
    2.Germany
    3.France
    '''
"""

messages = [("system",system_prompt)]
messages.append(("human",user_input))

response = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


### Chain of thought

In [50]:
234*45/3*342-4546*345

-367950.0

In [55]:
user_prompt = """
    Calculate/solve the equation and give the final result : 234*45/3*342-4546*345
    
    Your output should be in a structured json format exactly like the one below.You are not allowed to write anything other than the json object.
    
    {
        "steps":This is where you solve the equation bit by bit.You need to show your calculation for every step.Use BEDMAS rule.Feel free to write this in free text.
        "result":The final number resulted from calculating the equation given.
    }
    
"""

messages=[("human",user_prompt)]
response = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)

print(response)

{
    "steps": "First, we multiply 234 and 45: 234 * 45 = 10530. Then we divide the result by 3: 10530 / 3 = 3510. Next, we multiply 3510 by 342: 3510 * 342 = 1,203,420. After that, we multiply 4546 by 345: 4546 * 345 = 1,570,710. Finally, we subtract 1,570,710 from 1,203,420: 1,203,420 - 1,570,710 = -367,290.",
    "result": -367290
}


## RAG - Retireval Augmented Generation

In [56]:
user_prompt = """
    What's new in iphone 16?
    
"""

messages=[("human",user_prompt)]
response = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)

print(response)

Since the iPhone 16 has not been officially announced by Apple, I'll provide you with some rumored features and speculations based on industry trends and leaks. Keep in mind that these are not confirmed and may change when the actual device is released.

**Rumored Features:**

1. **Design:** The iPhone 16 is expected to have a similar design to the iPhone 15, with a stainless steel frame and a tougher glass back. However, some rumors suggest that it might have a slightly smaller notch or even a notchless design.
2. **Display:** The iPhone 16 is expected to feature a higher refresh rate display, possibly up to 120Hz or even 144Hz. This would provide a smoother and more responsive user experience.
3. **Camera:** The iPhone 16 is rumored to have a new periscope lens, which would allow for better zoom capabilities and a wider-angle lens. It might also feature a new telephoto lens with a longer focal length.
4. **Battery Life:** The iPhone 16 is expected to have a larger battery, which woul

In [57]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""


In [58]:
user_prompt = f"""
    {iphone_16}
    What's new in iphone 16?
    
"""

messages=[("human",user_prompt)]
response = get_chatbot_response(model_name=os.getenv("MODEL_NAME"),messages=messages)

print(response)

According to the provided information, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch display
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. Thinner bezels and a more durable Ceramic Shield.

3. Upgraded A18 chip (A18 Pro for Pro models) for improved performance, including:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max.

7. Switch to USB-C for faster charging and data transfer.

8. Support for up to 2x faster video encoding in Pr

In [59]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [60]:

data = [iphone_16,samsung_s23]

In [61]:

user_prompt = """What's new in iphone 16?"""

In [ ]:


def get_embeddings(texts:List[str]):
    embedding_model_name = "BAAI/bge-small-en-v1.5"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    hf_embeddings = HuggingFaceBgeEmbeddings(
        model_name=os.getenv("EMBEDDINGS_MODEL_NAME",embedding_model_name),
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder = os.getenv("SENTENCE_TRANSFORMERS_HOME")
    )
    embeddings_output = hf_embeddings.embed_documents(texts=texts)
    
    return embeddings_output
        
    
    




d:\Anaconda\envs\llmenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [77]:
user_prompt_embedding = get_embeddings(texts=[user_prompt])

In [78]:
len(user_prompt_embedding)

1

In [79]:
data_embeddings = get_embeddings(texts=data)

In [1]:
len(data_embeddings[0]) 

NameError: name 'data_embeddings' is not defined

In [82]:
data_similarity_scores = cosine_similarity(user_prompt_embedding,data_embeddings)

In [84]:
data_similarity_scores

array([[0.71307909, 0.46981073]])

In [85]:
closest_entry_index=data_similarity_scores.argmax()
closest_entry_index

0

In [86]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l